In [1]:
#Import dependencies
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import pymongo
import pandas as pd

In [2]:
# Import Splinter and set the chromedriver path
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Scrap the latest News
url="https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
News=soup.find('li', class_='slide')
news_title=News.find('div', class_="content_title").text
news_p = News.find('div', class_="article_teaser_body").text

In [4]:
#scrape lates image
url="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
Images=soup.find('li', class_='slide')
featured_image_url="https://www.jpl.nasa.gov"+Images.find('a')['data-fancybox-href']

In [5]:
# Scrape weather info
url="https://twitter.com/marswxreport?lang=en"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
Tweet=soup.find('li', class_='js-stream-item stream-item stream-item')
mars_weather=Tweet.find('div', class_="js-tweet-text-container").text

In [6]:
# Scrape facts about Mars
url="https://space-facts.com/mars/"
tables = pd.read_html(url)
Tabla=tables[0].to_html(index=False)
Tabla=Tabla.replace("\n","")

In [7]:
# Scrape Marse hemispheres data
url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
Hem=soup.find_all('div', class_='item')
#hemisphere_image_urls = {'title': [], 'img_url': []}
hemisphere_image_urls = []
    
for hem in Hem:
    url_n="https://astrogeology.usgs.gov"+hem.find('a')['href']
    browser.visit(url_n)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    hemisphere_image_urls.append({'title':soup.find('h2').text.replace(" Enhanced",""), 'img_url':soup.find_all('li')[0].find('a')['href']})
browser.quit()

In [8]:
## Conecting to MongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.Mission2Mars
collection = db.M2M


In [9]:
# Save scraped data to mangoDB 
collection.insert_one({'LN':{'LN_title':news_title, 'LN_text':news_p},
                        'weather':mars_weather,
                        'image':featured_image_url,
                        'facts':Tabla,
                        'hemispheres':hemisphere_image_urls})

In [14]:
print(collection.find_one())

{'_id': ObjectId('5d6768acb871af9d2e972b8a'), 'LN': {'LN_title': 'NASA Invites Students to Name Mars 2020 Rover', 'LN_text': "Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover."}, 'weather': '\nWe won’t be hearing from @MarsCuriosity or @NASAInSight for the next 2 weeks during Mars solar conjunction. Read more about why Mars missions go silent every 2 years: https://www.wral.com/mars-spacecraft-go-quiet-during-solar-conjunction/18595551/\xa0…pic.twitter.com/fWruE2v151\n', 'image': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA23345_hires.jpg', 'facts': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>Mars - Earth Comparison</th>      <th>Mars</th>      <th>Earth</th>    </tr>  </thead>  <tbody>    <tr>      <td>Diameter:</td>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>   